Syntax natürlicher Sprachen, WS 2024/25

# 08 - Übung

In [1]:
import nltk
from nltk.tree import Tree
from nltk.featstruct import FeatStruct
from nltk.featstruct import Feature, UnificationFailure, FeatStructReader
from sklearn.metrics import accuracy_score
import itertools

In [2]:
TYPE = nltk.featstruct.TYPE

def check_sanity_constraints(th):
    for type1, type2 in itertools.product(th, th):
        if type1 in th[type2] and type2 in th[type1]:
            if type1 != type2:
                raise ValueError(
                    "The type hierarchy is not antisymmetric! " +
                    "{} subsumes {} and vice versa!".format(
                        type1, type2
                    )
                )

def refl_trans_closure(type_hierarchy):
    # make everything a set
    # and compute reflexive closure
    closure = {}
    for t in type_hierarchy:
        closure[t] = set(type_hierarchy[t])
        closure[t].add(t)

    # compute transitive closure
    still_changes = True
    while still_changes:
        still_changes = False
        for x in closure:
            new_for_x = set()
            for y in closure[x]:
                for z in closure[y]:
                    new_for_x.add(z)
            len_before = len(closure[x])
            closure[x].update(new_for_x)
            still_changes |= len(closure[x]) > len_before

    return closure

class HierarchicalFeature(Feature):
    def __init__(self, name, type_hierarchy, **kwargs):
        super(HierarchicalFeature, self).__init__(name, **kwargs)

        self.hierarchy = refl_trans_closure(type_hierarchy)
        check_sanity_constraints(self.hierarchy)

    def unify_base_values(self, fval1, fval2, bindings):
        candidates = self.hierarchy[fval1].intersection(self.hierarchy[fval2])
        score = {t: 0 for t in candidates}
        for type1, type2 in itertools.product(candidates, candidates):
            if type1 in self.hierarchy[type2]:
                score[type1] += 1

        return min(candidates, key=score.__getitem__, default=UnificationFailure)

## Aufgabe 1 - Kasusrektion, Agreement und Subkategorisierung

### Welche der folgenden morphosyntaktischen Beschränkungen ist in den unteren Sätzen jeweils verletzt?

1. **Kasusrektion**
2. **Agreement** (nominale oder verbale Kongruenz)
3. **Subkategorisierung** (Anzahl und Art der verbalen Argumente)

(i)  *Ich beobachtet den Mond.*

In [83]:
#

(ii)  *Ich beobachte dem Mond.*

In [ ]:
#

(iii)  *Ich beobachte das Monde.*

In [ ]:
#

(iii)  *Ich beobachte ihm den Mond.*

In [ ]:
#

## Aufgabe 2 - Unifikation und Subsumption


### a) Stehen die beiden folgenden Merkmalstrukturen in einer Subsumptions- und/oder Unifikationsbeziehung? Wenn ja, geben Sie das Ergebnis der Subsumption bzw. Unifikation an, erläutern Sie andernfalls:


In [3]:
f0 = FeatStruct("[AGR=[NUM=sg]]")
f1 = FeatStruct("[AGR=[NUM=sg, PERS=3]]")

In [ ]:
#Subsumption (f0 ⊑ f1)
f0.subsumes(f1)

In [ ]:
#Subsumption (f1 ⊑ f0)
f1.subsumes(f0)

In [ ]:
#Unification (f0 ⊔ f1)
print(f0.unify(f1))

### b) Stehen die beiden folgenden Merkmalstrukturen in einer Subsumptions- und/oder Unifikationsbeziehung? Wenn ja, geben Sie das Ergebnis der Subsumption bzw. Unifikation an, erläutern Sie andernfalls:


In [4]:
f0 = FeatStruct("[]")
f1 = FeatStruct("[AGR=[NUM=sg, PERS=3]]")

In [ ]:
#Subsumption (f0 ⊑ f1)
f0.subsumes(f1)

In [ ]:
#Subsumption (f1 ⊑ f0)
f1.subsumes(f0)

In [ ]:
#Unification (f0 ⊔ f1)
print(f0.unify(f1))

### c) Stehen die beiden folgenden Merkmalstrukturen in einer Subsumptions- und/oder Unifikationsbeziehung? Wenn ja, geben Sie das Ergebnis der Subsumption bzw. Unifikation an, erläutern Sie andernfalls:


In [5]:
f0 = FeatStruct("[AGR=[NUM=sg, CASE=nom]]") 
f1 = FeatStruct("[AGR=[NUM=sg, PERS=3]]") 

In [ ]:
#Subsumption (f0 ⊑ f1)
f0.subsumes(f1)

In [ ]:
#Subsumption (f1 ⊑ f0)
f1.subsumes(f0)

In [ ]:
#Unification (f0 ⊔ f1)
print(f0.unify(f1))

### d) Stehen die beiden folgenden Merkmalstrukturen in einer Subsumptions- und/oder Unifikationsbeziehung? Wenn ja, geben Sie das Ergebnis der Subsumption bzw. Unifikation an, erläutern Sie andernfalls:


In [6]:
f0 = FeatStruct("[AGR=[NUM=pl]]") 
f1 = FeatStruct("[AGR=[NUM=sg, PERS=3]]")

In [ ]:
#Subsumption (f0 ⊑ f1)
f0.subsumes(f1)

In [ ]:
#Subsumption (f1 ⊑ f0)
f1.subsumes(f0)

In [ ]:
#Unification (f0 ⊔ f1)
print(f0.unify(f1))

## Aufgabe 3 - Unifikation

#### Gegeben seien folgende Merkmalstrukturen:

In [7]:
f1 = FeatStruct(
    '[Vorname=Max, Nachname=Mustermann,' + 
    'Privat=[Strasse=Hauptstrasse, Ort=[Stadt=Muenchen]]]'
)
f2 = FeatStruct(
    '[Arbeit=[Strasse="Oettingenstrasse", Ort=(1)[Stadt=Muenchen]],' +
    'Privat=[Ort->(1)]]')
f3 = FeatStruct(
    '[Strasse="Hauptstrasse"]'
)
f4 = FeatStruct(
    '[Privat=[Strasse="Hauptstrasse", Ort=[Stadt=Passau]]]'
)

### a) Unifizieren f1 und f2? Gegeben Sie ggf. das Ergebnis an.

In [8]:
print(f1)

[ Nachname = 'Mustermann'                         ]
[                                                 ]
[            [ Ort     = [ Stadt = 'Muenchen' ] ] ]
[ Privat   = [                                  ] ]
[            [ Strasse = 'Hauptstrasse'         ] ]
[                                                 ]
[ Vorname  = 'Max'                                ]


In [9]:
print(f2)

[          [ Ort     = (1) [ Stadt = 'Muenchen' ] ] ]
[ Arbeit = [                                      ] ]
[          [ Strasse = 'Oettingenstrasse'         ] ]
[                                                   ]
[ Privat = [ Ort -> (1) ]                           ]


In [ ]:
#### Mit der Ausführung des print-Statements können Sie abschließend die Lösung einsehen:
print(f1.unify(f2))

### b) Unifizieren f2 und f4? Gegeben Sie ggf. das Ergebnis an.

In [10]:
print(f2)

[          [ Ort     = (1) [ Stadt = 'Muenchen' ] ] ]
[ Arbeit = [                                      ] ]
[          [ Strasse = 'Oettingenstrasse'         ] ]
[                                                   ]
[ Privat = [ Ort -> (1) ]                           ]


In [11]:
print(f4)

[          [ Ort     = [ Stadt = 'Passau' ] ] ]
[ Privat = [                                ] ]
[          [ Strasse = 'Hauptstrasse'       ] ]


In [ ]:
# 

In [ ]:
#### Mit der Ausführung des print-Statements können Sie abschließend die Lösung einsehen:
print(f2.unify(f4))

## Aufgabe 4 - Typhierarchie

#### Gegeben sei folgende Typhierarchie, die ein kombiniertes Person-Numerus-Agreement-Feature als hierarchisch durch Subsumptionsbeziehung gegliederte Featurewerte  modelliert ([1] ⊑ [1sg] usw.).


In [12]:
type_hierarchy = {
    "1": ["1sg","1pl"],
    "2": ["2sg", "2pl"],
    "3": ["3sg", "3pl"],
    "sg": ["1sg", "2sg", "3sg"],
    "pl": ["1pl", "2pl", "3pl"],
    "1sg": [],
    "1pl": [],
    "2sg": [],
    "2pl": [],
    "3sg": [],
    "3pl": [],
}
AGR = HierarchicalFeature("AGR", type_hierarchy)
reader = FeatStructReader(features=(AGR,))

### a) Geben Sie eine (nicht-leere) Merkmalstruktur `f1` aus der Typhierarchie an, sodass gilt:

`f1` subsumiert `f0`.

In [ ]:
f0 = reader.fromstring("[*AGR*='1sg']") #strings containing numbers have to be marked as strings
f1 = reader.fromstring("[]") # empty featstruct subsumes everything
f1.subsumes(f0)

In [ ]:
f0.subsumes(f1)

In [ ]:
print(f0.unify(f1))

### b) Geben Sie zwei (nicht-leere) Merkmalstrukturen `f1` und `f2` aus der Typhierarchie an, sodass gilt:

- `f1` subsumiert NICHT `f0`
- `f0` subsumiert  NICHT `f1`
- aber `f1` und `f0` sind unifizierbar.

In [ ]:
f0 = reader.fromstring("[]")
f1 = reader.fromstring("[]")
print(f1.subsumes(f0), f0.subsumes(f1))

#### Unifikation als kleinste obere Schranke in der Subsumptionsbeziehung:

In [ ]:
print(f0.unify(f1))

### c) Geben Sie zwei (nicht-leere) Merkmalstrukturen `f1` und `f2` aus der Typhierarchie an, sodass gilt:

- `f1` subsumiert NICHT `f0`
- `f0` subsumiert  NICHT `f1`
- und `f1` und `f0` sind NICHT unifizierbar.

In [ ]:
f0 = reader.fromstring("[]")
f1 = reader.fromstring("[]")
print(f1.subsumes(f0), f0.subsumes(f1))

In [ ]:
print(f0.unify(f1))

## *Aufgabe 5 - Getypte Merkmalstrukturen in CFG*

#### Gegeben sei folgende Typhierarchie:

$$\bot \sqsubseteq \text{Genitiv}$$
$$\bot \sqsubseteq \text{nicht-Genitiv}$$
$$\text{nicht-Genitiv} \sqsubseteq \text{Nominativ-Akkusativ}$$
$$\text{nicht-Genitiv} \sqsubseteq \text{Dativ}$$
$$\text{Nominativ-Akkusativ} \sqsubseteq \text{Nominativ}$$
$$\text{Nominativ-Akkusativ} \sqsubseteq \text{Akkusativ}$$


### a) Implementieren Sie ein Feature `CASE`, das der vorgegebenen Typhierarchie entspricht.

#### Hier muss die Typhierarchie in Form eines Dictionary definiert werden:

In [16]:
type_hierarchy = {
    
}

In [17]:
CASE = HierarchicalFeature("CASE", type_hierarchy)
reader = FeatStructReader(features=(CASE,))

### b) Geben Sie eine (nicht-leere) Merkmalstruktur `f1`  aus der Typhierarchie an, sodass gilt:

`f1` subsumiert NICHT `f0`.

In [18]:
f0 = reader.fromstring("[*CASE*=NomAkk]")
f1 = reader.fromstring("[]")
f1.subsumes(f0)

True

In [ ]:
f0.subsumes(f1)

In [ ]:
print(f0.unify(f1))

### c) Nutzen Sie das getypte `CASE`-Feature nun, um Übergenerierung in folgender Grammatik zu vermeiden:

In [ ]:
grammar = """
S -> NP VP
NP -> DET NOM
NOM -> N NP | N
VP -> V

V -> "schläft"
DET[GEN=mask] -> "der" | "des"
DET[GEN=fem] -> "die" | "der"
DET[GEN=neut] -> "das" | "des"
N[GEN=mask] -> "Mann" | "Mannes"
N[GEN=fem] -> "Frau"
N[GEN=neut] -> "Kind" | "Kindes"
"""

In [ ]:
CASE = HierarchicalFeature("CASE", type_hierarchy)
compiled_grammar = nltk.grammar.FeatureGrammar.fromstring(
    grammar, features=(CASE, TYPE)
)
parser = nltk.FeatureEarleyChartParser(compiled_grammar)

#### Folgendes sollte funktionieren:

In [ ]:
for t in parser.parse("das Kind des Mannes schläft".split()):
    t = Tree.fromstring(str(t).replace(", ",","))
#    t.pretty_print(unicodelines=True)
    display(t)

#### Folgendes sollte leer sein:

In [ ]:
list(parser.parse("des Mannes schläft".split()))